In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = None

In [2]:
df = pd.read_csv("datos/df_interesantes.csv", index_col = 0)


**Pair Programming Limpieza II**

Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**:

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?

- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?

- Independientemente de la edad, sufren los hombres más ataques que las mujeres?

- ¿En qué mes ocurren más ataques?

- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?

- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex.

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra.

Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

1. Columna de species:

Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna.

Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

Imaginemos que el valor de una celda es el siguiente:
​
'White shark, 3.5 m'
​
Tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
Un patrón que podriamos usar es: 
​
patron_blanco = r".*[Ww](hite|HITE).*" (esto podría ser así porque puede estar en mayúsculas o en minúsculas.)
​
De la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar".

Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [3]:
df.head()

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD
0,2018,usa,F,57,White shark,NaN,N
1,2018,brazil,M,18,Tiger shark,NaN,N
2,2018,usa,M,15,"Bull shark, 6'",NaN,N
3,2018,australia,M,32,Grey reef shark,NaN,N
4,2018,england,M,21,Invalid incident,NaN,N


In [4]:
patron_white = r".*[Ww](hite|HITE).*"
patron_tiger = r".*[Tt](iger|IGER).*"
patron_grey = r".*[Gg](rey|REY).*"
patron_lemon = r".*[Ll](emon|EMON).*"
patron_bull = r".*[Bb](ull|ULL).*"

In [5]:
def especies(col):
        patron_white = re.findall (".*[Ww](hite|HITE).*", str(col))
        patron_tiger = re.findall (".*[Tt](iger|IGER).*", str(col))
        patron_grey = re.findall (".*[Gg](rey|REY).*", str(col))
        patron_lemon = re.findall (".*[Ll](emon|EMON).*", str(col))
        patron_bull = re.findall (".*[Bb](ull|ULL).*", str(col))

        if len(patron_white) > 0:
            return "White"
        elif len(patron_tiger) > 0:
            return "Tiger"
        elif len(patron_grey) > 0:
            return "Grey"
        elif len (patron_lemon) > 0:
            return "Lemon"
        elif len (patron_bull) > 0:
            return "Bull"
        else:
            return "Desconocido"
    

In [6]:
df ["nueva_especie"] = (df["SPECIES_"].apply(especies))
df ["nueva_especie"].unique()

array(['White', 'Tiger', 'Bull', 'Grey', 'Desconocido', 'Lemon'],
      dtype=object)


2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

- Edad en formato string.

- Edades separadas por &, or, to, >

- Edades con ?


Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:

- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

Puede que os salga un error similar a este:

TypeError: expected string or bytes-like object

Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

df['nombre_columna'] = df['nombre_columna'].astype(str)

Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.

- Por último cambiad el tipo de la columna de string a integer.

In [7]:
df['AGE'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '60s',
       '51', '61', '59', '42', '6', '27', '23', '29', '39', '24', '12',
       '26', '71', '43', '9', '44', '14', '62', '52', '38', '68', '16',
       '47', '63', '70', '41', '40s', '53', '20s', '7', '66', '45', '74',
       '64', '8', '56', '49', '18 or 20', 'Teen', '30s', '77', '8 or 10',
       '84', '\xa0 ', '_', '30 or 36', '6½', '5', ' 30', ' 28', "60's",
       '67', '>50', '? & 19', '21, 34,24 & 35', '30 & 32', '13 or 18',
       '7 or 8', '9 or 10', nan, 'young', '13 or 14'], dtype=object)

In [8]:
patron_numeros = r"(\d\d)"

In [9]:
def numeros (col):
    lista = re.findall (patron_numeros, str(col))

    if len(lista) > 0:
        return str(lista [0])
    

In [10]:
df["edad_limpia"] = df["AGE"].apply(numeros)

In [11]:
df["edad_limpia"].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '51',
       '61', '59', '42', None, '27', '23', '29', '39', '24', '12', '26',
       '71', '43', '44', '14', '62', '52', '38', '68', '16', '47', '63',
       '70', '41', '53', '66', '45', '74', '64', '56', '49', '77', '84',
       '67'], dtype=object)

In [12]:
df.dtypes

YEAR              int64
COUNTRY          object
SEX_             object
AGE              object
SPECIES_         object
fecha            object
FATALIDAD        object
nueva_especie    object
edad_limpia      object
dtype: object

In [13]:
#edades_divididas = df.explode("edad_limpia", ignore_index=False)
#edades_divididas.sample(12)

#hemos comentado esta celda para ver si es por esto por lo que no nos deja hacer el astype, pero los resultados son los mismos con ella que sin ella

In [14]:
df['edad_limpia'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '51',
       '61', '59', '42', None, '27', '23', '29', '39', '24', '12', '26',
       '71', '43', '44', '14', '62', '52', '38', '68', '16', '47', '63',
       '70', '41', '53', '66', '45', '74', '64', '56', '49', '77', '84',
       '67'], dtype=object)

In [15]:
df.dtypes

YEAR              int64
COUNTRY          object
SEX_             object
AGE              object
SPECIES_         object
fecha            object
FATALIDAD        object
nueva_especie    object
edad_limpia      object
dtype: object

In [16]:
#intentamos cambiar tipo con este metodo pero no lo conseguimos. 
df["edad_limpia"]=df["edad_limpia"].astype( 'int64', errors='raise', copy=True )

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
#tampoco con este y además aparecen nones que antes no estaban
df["edad_limpia"].astype(np.number)
df["edad_limpia"].unique

<bound method Series.unique of 0         57
1         18
2         15
3         32
4         21
        ... 
1649    None
1657    None
1661    None
1670    None
1671      15
Name: edad_limpia, Length: 1620, dtype: object>

In [17]:
pd.to_numeric (df['edad_limpia'])

0       57.0
1       18.0
2       15.0
3       32.0
4       21.0
        ... 
1649     NaN
1657     NaN
1661     NaN
1670     NaN
1671    15.0
Name: edad_limpia, Length: 1620, dtype: float64

In [ ]:
# parece que ha cambiado pero ahora es float y hay NaNs
# pasamos al siguiente pair con las edades así por recomendacion de los profesores para poder seguir avanzando.

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [18]:
# pasamos al siguiente pair con las edades en float por recomendacion de los profesores para poder seguir avanzando.
df.to_csv("datos/limpieza_dos.csv")
